<a href="https://colab.research.google.com/github/snpushpi/Covid19_Contact_tracing_repo_beta/blob/master/UCI_Final_No_Fake_path_contact_tracing_different_hash_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [29]:
import pandas as pd
import os

In [30]:
from google.colab import files
uploaded = files.upload()

Saving Raw_Dataset1.csv to Raw_Dataset1.csv


In [31]:
import io

In [32]:
new_dataframe= pd.read_csv(io.BytesIO(uploaded['Raw_Dataset1.csv']))

In [33]:
new_dataframe[:20]

,Unnamed: 0,Unnamed: 0.1,latitude,lognitude,timestamp,Unnamed: 3,Unnamed: 4,ID
0,0,0,4147.5440,8806.6697,11:33:39,197.886254,353.603657,0
1,1,1,4147.5426,8806.6695,11:33:42,197.886430,353.602047,0
2,2,2,4147.5416,8806.6676,11:33:50,197.888069,353.600901,0
3,3,3,4147.5417,8806.6676,11:33:52,197.888068,353.601016,0
4,4,4,4147.5417,8806.6677,11:33:54,197.887982,353.601016,0
5,5,5,4147.5418,8806.6677,11:33:55,197.887982,353.601131,0
6,6,6,4147.5570,8806.6822,11:34:04,197.875456,353.618582,0
7,7,7,4147.5573,8806.6833,11:34:05,197.874508,353.618925,0
8,8,8,4147.5630,8806.6932,11:34:10,197.865969,353.625459,0
9,9,9,4147.5683,8806.7092,11:34:16,197.852181,353.631520,0


create a dataframe object from the csv file

Next few parts, we will be making our data in a usable format for feeding into our dataset

In [34]:
def list_generate(new_dataframe,user_num):    
    user_lat = new_dataframe[new_dataframe.ID==user_num]['latitude'].to_list()
    user_lon = new_dataframe[new_dataframe.ID==user_num]['lognitude'].to_list()
    user_timestamp = new_dataframe[new_dataframe.ID==user_num]['timestamp'].to_list()
    l = len(new_dataframe[new_dataframe.ID==user_num])
    user_path = [(user_lat[i], user_lon[i], user_timestamp[i]) for i in range(l)]
    return user_path


In [35]:
len(new_dataframe[new_dataframe.ID==0])

1123

For 50 infected users, we will create a list of lists where each list is the path of one infected patient

In [36]:
querier_path = list_generate(new_dataframe,0)
infected_list = []
for i in range(1,19):
    infected_list.append(list_generate(new_dataframe,i))
    

In [37]:
min_lat = 4145.0
max_lat = 4152.612
min_lon = 8737.9052
max_lon = 8812.6234

In [49]:
def position_determiner(lat, lon):
    '''Given a latitude, longitude co-ordinate, we have to find the small universe it belongs to.'''

    k = math.floor((lat-min_lat)/0.0030)
    j = math.floor((lon-min_lon)/0.0030)
    return (k,j)
    

In [40]:
def path_divider(path):
    '''given a path (a list of tuples, (lat, lon, timestamp)), divides it into zones.
    returns: a dictionary zone->[{timestamp->(lat,lon, timestamp)},[timestamp_list]]'''
    track_dict = {}
    for elt in path:
        temp_val = position_determiner(elt[0], elt[1])
        if temp_val not in track_dict:
            track_dict[temp_val]=[{},[]]
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
        else:
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
    return track_dict


In [41]:
class Querier():
    def __init__(self,true_path):
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.intersection = []

    def filter_divider(self):
        '''returns zone->timestamp_list dictionary.'''
        self.filtered_dictionary = path_divider(self.true_path)     
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict

    def checker(self, freq_dict, zone, T):
        '''given the dictionary, checks the intersection with its elements, in other words, for each of its elements in this zone,
        it checks whether the frequency in the dictionary is greater than 1.'''
        timestamp_list = self.filtered_dictionary[zone][1]
        new_all_list = [zone+(timestamp,) for timestamp in timestamp_list]
        list_to_return = []
        for elt in new_all_list:
            if elt in freq_dict:
                if math.floor(freq_dict[elt])>=T:
                    self.intersection.append(elt)
                    list_to_return.append(elt[2])
        return list_to_return
                
    def announcer(self):
        '''returns the self.intersection list.'''
        result_list=[]
        for elt in self.intersection:
            result_list.append((elt[0]*0.0030+min_lat,elt[1]*0.0030+min_lon,elt[2]))
        return result_list


In [42]:
class Infected():
    def __init__(self, true_path,width, depth,epsilon):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.new_level2_filter = {}
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
    #we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
    #and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
    #result vector. 

    def Client_Side(self,d, seed):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def send_flipped_bit_vector(self,zone,seed):
        result_list = []
        for elt in self.new_level2_filter[zone]:
            result_list.append(self.Client_Side(elt,seed))
        return result_list

    def filter_divider(self):
        '''returns: zone->timestamp_list.'''
        self.filtered_dictionary = path_divider(self.true_path)
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict
        
    def filter2(self,filtered_dict_from_server):
        '''filtered_dict_from_server, a dictionary with zone->timestamps.
        modifies: a new_level2_filter list, and new_level2_filter_list[zone] is a list of (lat, lon, timestamp).'''
        for zone in filtered_dict_from_server:
            self.new_level2_filter[zone]=[]
            for elt in filtered_dict_from_server[zone]:
                self.new_level2_filter[zone].append(self.filtered_dictionary[zone][0][elt])



In [43]:
class Server():
    def __init__(self, width, depth, epsilon):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.zone_list=[]
        self.new_filtered_list = []

    def Compute_Sketch_Matrix(self,D, seed):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def frequency_counter(self, Sketch_Matrix, total_path_length, universe_list, seed):
        ''' we get a sketch matrix and we try to get the frequencies for all elements in the true_path.'''
        frequency_dict = {}
        for d in universe_list:
            n = total_path_length
            frac1 = self.width/(self.width-1)
            frac2 = n/self.width
            row_sum = 0
            for i in range(self.depth):
                hex_num = hashlib.sha256((str(seed[i])+str(d)).encode('utf-8')).hexdigest()
                int_hex = int(hex_num[:5],16)
                index = int_hex % self.width
                row_sum = row_sum + Sketch_Matrix[i,index]
            avg_row_sum = row_sum/self.depth 
            subtraction = avg_row_sum - frac2 
            assumed_freq = frac1*subtraction 
            frequency_dict[d]=assumed_freq
        return frequency_dict

    def checker(self,querier_filtered_path, infected_patients_filtered_dict_list):
        '''gets zone-> timestamps and returns list of filtered zone-> set(timestamps) dictionaries.'''
        l = len(infected_patients_filtered_dict_list)
        for i in range(l):
            temp_dict={}
            for zone in infected_patients_filtered_dict_list[i]:
                if zone in querier_filtered_path:
                    temp_dict[zone] = set(infected_patients_filtered_dict_list[i][zone]) & set(querier_filtered_path[zone])
            self.new_filtered_list.append(temp_dict)#For all infected patients, it maintains a dictionary, mapping zone to timestamps.
        return self.new_filtered_list
    
    def universe_generate(self, zone, timestamp_list):
        '''generate the universe list with the timestamps.'''
        lat = zone[0]
        lon = zone[1]
        lat_list = []
        lon_list = []
        universe_list = []
        for i in np.arange(lat, lat+0.0031, 0.0001):
            for j in np.arange(lon, lon+0.0031, 0.0001):
                for time in timestamp_list:
                    universe_list.append((i, j, time))
        return universe_list

    def bit_flip_caller(self, infected_patients, querier, run_number, T):
        '''Will call the flip bit method on all zones of each infected patient.'''
        zone_universe_list = set()
        l = len(self.new_filtered_list)
        for i in range(l):
            zone_universe_list|=set(self.new_filtered_list[i].keys())
        big_timestamp_dictionary = {e:set() for e in zone_universe_list}
        big_tracker_dictionary = {e:[] for e in zone_universe_list}#zone-> infected person who is in that zone
        #generating zone->timestamps for all infected patient dictionary.
        for zone in zone_universe_list:
            for i in range(l):
                if zone in self.new_filtered_list[i]:
                    big_timestamp_dictionary[zone]|=set(self.new_filtered_list[i][zone])
                    big_tracker_dictionary[zone].append(i)
        
        intersection_check_dictionary = {zone:[] for zone in zone_universe_list}
        for zone in zone_universe_list:
            universe_list = self.universe_generate(zone, big_timestamp_dictionary[zone])
            freq_cum_dict = {d:0 for d in universe_list}
            for i in range(run_number):
                big_vect_list = []
                seed = np.random.choice(200, size=150, replace= False)
                #call a function which does some calculation
                for j in big_tracker_dictionary[zone]:
                    big_vect_list+=infected_patients[j].send_flipped_bit_vector(zone,seed)
                n = len(big_vect_list)
                M = self.Compute_Sketch_Matrix(big_vect_list,seed)
                freq_dict = self.frequency_counter(M,n, universe_list,seed)
                for elt in freq_dict:
                    freq_cum_dict[elt]=(i*freq_cum_dict[elt]+freq_dict[elt])/(i+1)
                intersection_check_dictionary[zone].append(querier.checker(freq_cum_dict, zone, T))
        result_list = []
        for i in range(run_number):
            temp_list = []
            for zone in zone_universe_list:
                temp_list+=intersection_check_dictionary[zone][i]
            result_list.append(temp_list)
        final_result_list = []
        final_result_list.append(result_list[0])
        for i in range(1,run_number):
            temp_list = result_list[i]+final_result_list[i-1]
            final_result_list.append(temp_list)
        return final_result_list
        


In [ ]:
#in the new mode, we invoke this method and the server receives this flipped bit vectors, as we want to average the 
#bit flipping thing, we run this process a mentioned number of times. and make compute sketch matrix.

In [44]:
def driver_code(width, depth, epsilon, run_number, T):
    '''1. Create instances of querier, server and infected patients.
       2. Call the filter method on all the participants in the server, so all the querier and the infectd patients filter out their paths 
          in multiple universes. This function will output a dictionary kepys mapping to 2 lists, one list is the co-ordinates in that universe and another list is the timestamp.
       3. Now send this filtered timestamp and zone list to the server. Form can be (zone code->timestamps in that zone)
       4. Taking the querier's timestamp against the infected patient's timestamp, server will level2 filtering and send similar form of 
          dictionary to all the infected patients and the querier, so basically finding the timestamp intersection between querier and infected patient pair
          and server is keeping the lists for his calculation.
       5. The participants, upon receiving the information, filters out the information further and start processing data from their end, aka, 
          calls the server caller method.(So in our implementation, we will be processing all infected patients one by one)
       6. The server caller method, is a common method which will be used in other parts too, when this method is called from some 
          infected patient, it asks for flipped vectors a mentioned number of times from the infected patient and upon receiving it every time,
          the server does the compute sketch matrix operation and contnues averaging it and finally gets the aggregated sketch matrix, as the server knows which 
          infected patient and from which zone, it was being called, he finally calls freq generator with this information.(Zone code, sketch matrix and server has saved the timestamp info
          for generating universe list)
       7. Thus for each caller method, server keeps generating frequency histogram and keeps aggregating it and will finally send it to the querier 
       
    '''
    querier = Querier(querier_path)
    infected_patients = []
    for infected in infected_list:
        infected_patients.append(Infected(infected, width, depth, epsilon))
    server_instance = Server(width, depth, epsilon)
    infected_patients_filtered_dict_list=[]
    for e in infected_patients:
        infected_patients_filtered_dict_list.append(e.filter_divider())
    querier_filter = querier.filter_divider()
    new_filtered_list = server_instance.checker(querier_filter, infected_patients_filtered_dict_list)
    l = len(new_filtered_list)
    for i in range(l):
        infected_patients[i].filter2(new_filtered_list[i])
    tracker = server_instance.bit_flip_caller(infected_patients, querier, run_number, T)
    return tracker, querier.announcer()



In [45]:
ground_truth_timestamp_coordinates={' 11:38:29 ', ' 11:37:01 ', ' 11:36:22 ', ' 11:35:25 ', ' 11:46:08 ', ' 11:45:10 ', ' 11:46:34 ', ' 11:35:29 ', ' 11:45:29 ', ' 11:38:27 ', ' 11:46:17 ', ' 11:36:35 ', ' 11:45:45 ', ' 11:46:43 ', ' 11:46:04 ', ' 11:45:18 ', ' 11:38:49 ', ' 11:45:27 ', ' 11:42:32 ', ' 11:33:54 ', ' 11:39:04 ', ' 11:38:36 ', ' 11:38:21 ', ' 11:38:31 ', ' 11:36:21 ', ' 11:45:21 ', ' 11:43:02 ', ' 11:46:21 ', ' 11:42:24 ', ' 11:38:42 ', ' 11:36:44 ', ' 11:41:54 ', ' 11:46:26 ', ' 11:36:48 ', ' 11:45:41 ', ' 11:38:24 ', ' 11:42:36 ', ' 11:36:55 ', ' 11:39:00 ', ' 11:46:01 ', ' 11:38:19 ', ' 11:42:30 ', ' 11:38:56 ', ' 11:42:13 ', ' 11:34:28 ', ' 11:43:01 ', ' 11:42:56 ', ' 11:39:11 ', ' 11:42:46 ', ' 11:36:56 ', ' 11:42:45 ', ' 11:46:28 ', ' 11:46:03 ', ' 11:42:02 ', ' 11:45:24 ', ' 11:36:14 ', ' 11:41:45 ', ' 11:39:16 ', ' 11:41:49 ', ' 11:37:11 ', ' 11:45:42 ', ' 11:41:44 ', ' 11:34:17 ', ' 11:34:16 ', ' 11:33:39 ', ' 11:39:10 ', ' 11:45:56 ', ' 11:45:17 ', ' 11:41:43 ', ' 11:36:37 ', ' 11:38:17 ', ' 11:46:29 ', ' 11:34:10 ', ' 11:36:45 ', ' 11:36:40 ', ' 11:45:54 ', ' 11:42:43 ', ' 11:42:04 ', ' 11:42:00 ', ' 11:38:48 ', ' 11:36:25 ', ' 11:42:06 ', ' 11:38:44 ', ' 11:46:12 ', ' 11:46:33 ', ' 11:42:31 ', ' 11:46:31 ', ' 11:37:08 ', ' 11:42:54 ', ' 11:45:58 ', ' 11:38:30 ', ' 11:42:09 ', ' 11:33:52 ', ' 11:42:58 ', ' 11:36:46 ', ' 11:42:37 ', ' 11:45:37 ', ' 11:45:46 ', ' 11:41:51 ', ' 11:39:02 ', ' 11:38:40 ', ' 11:39:07 ', ' 11:36:20 ', ' 11:39:14 ', ' 11:45:12 ', ' 11:35:23 ', ' 11:36:26 ', ' 11:45:35 ', ' 11:42:50 ', ' 11:45:28 ', ' 11:45:09 ', ' 11:36:19 ', ' 11:38:59 ', ' 11:46:10 ', ' 11:45:26 ', ' 11:38:20 ', ' 11:45:39 ', ' 11:42:29 ', ' 11:38:43 ', ' 11:46:41 ', ' 11:45:30 ', ' 11:34:27 ', ' 11:41:56 ', ' 11:34:04 ', ' 11:42:53 ', ' 11:42:40 ', ' 11:42:27 ', ' 11:35:28 ', ' 11:42:05 ', ' 11:34:26 ', ' 11:42:34 ', ' 11:37:02 ', ' 11:42:17 ', ' 11:46:38 ', ' 11:46:46 ', ' 11:42:14 ', ' 11:41:48 ', ' 11:42:11 ', ' 11:39:01 ', ' 11:39:19 ', ' 11:41:55 ', ' 11:41:58 ', ' 11:36:50 ', ' 11:45:11 ', ' 11:36:15 ', ' 11:46:19 ', ' 11:46:07 ', ' 11:45:38 ', ' 11:45:44 ', ' 11:42:38 ', ' 11:41:40 ', ' 11:42:10 ', ' 11:42:33 ', ' 11:45:59 ', ' 11:38:54 ', ' 11:34:05 ', ' 11:46:35 ', ' 11:46:09 ', ' 11:46:25 ', ' 11:36:18 ', ' 11:38:51 ', ' 11:42:21 ', ' 11:37:05 ', ' 11:46:40 ', ' 11:42:51 ', ' 11:39:09 ', ' 11:46:45 ', ' 11:42:35 ', ' 11:42:03 ', ' 11:38:53 ', ' 11:45:47 ', ' 11:46:24 ', ' 11:36:38 ', ' 11:38:58 ', ' 11:36:28 ', ' 11:46:00 ', ' 11:36:51 ', ' 11:39:12 ', ' 11:46:44 ', ' 11:36:23 ', ' 11:45:16 ', ' 11:45:20 ', ' 11:45:57 ', ' 11:42:25 ', ' 11:45:49 ', ' 11:46:32 ', ' 11:42:08 ', ' 11:36:42 ', ' 11:46:27 ', ' 11:45:31 ', ' 11:46:37 ', ' 11:38:45 ', ' 11:39:13 ', ' 11:38:25 ', ' 11:38:37 ', ' 11:39:15 ', ' 11:34:32 ', ' 11:38:18 ', ' 11:35:33 ', ' 11:38:32 ', ' 11:35:27 ', ' 11:36:57 ', ' 11:42:22 ', ' 11:36:27 ', ' 11:46:16 ', ' 11:33:42 ', ' 11:38:57 ', ' 11:45:40 ', ' 11:39:05 ', ' 11:35:34 ', ' 11:42:23 ', ' 11:42:18 ', ' 11:42:01 ', ' 11:39:06 ', ' 11:36:13 ', ' 11:36:12 ', ' 11:38:50 ', ' 11:37:06 ', ' 11:36:58 ', ' 11:41:59 ', ' 11:39:03 ', ' 11:36:52 ', ' 11:38:23 ', ' 11:45:43 ', ' 11:42:47 ', ' 11:41:42 ', ' 11:45:25 ', ' 11:36:30 ', ' 11:36:34 ', ' 11:42:52 ', ' 11:38:22 ', ' 11:42:42 ', ' 11:41:46 ', ' 11:38:26 ', ' 11:35:26 ', ' 11:39:18 ', ' 11:36:17 ', ' 11:38:38 ', ' 11:35:22 ', ' 11:34:25 ', ' 11:35:30 ', ' 11:34:33 ', ' 11:34:40 ', ' 11:41:41 ', ' 11:45:08 ', ' 11:36:39 ', ' 11:41:52 ', ' 11:36:24 ', ' 11:38:28 ', ' 11:45:52 ', ' 11:43:04 ', ' 11:38:39 ', ' 11:36:43 ', ' 11:46:02 ', ' 11:35:31 ', ' 11:46:20 ', ' 11:42:15 ', ' 11:38:46 ', ' 11:42:28 ', ' 11:43:03 ', ' 11:37:09 ', ' 11:45:14 ', ' 11:42:20 ', ' 11:46:36 ', ' 11:38:41 ', ' 11:42:26 ', ' 11:42:48 ', ' 11:45:48 ', ' 11:45:51 ', ' 11:46:47 ', ' 11:42:44 ', ' 11:33:55 ', ' 11:45:32 ', ' 11:41:50 ', ' 11:45:50 ', ' 11:46:23 ', ' 11:42:39 ', ' 11:38:35 ', ' 11:45:36 ', ' 11:36:54 ', ' 11:41:57 ', ' 11:38:55 ', ' 11:36:16 ', ' 11:45:55 ', ' 11:42:55 ', ' 11:46:06 ', ' 11:35:35 ', ' 11:33:50 ', ' 11:46:30 ', ' 11:42:12 ', ' 11:36:49 ', ' 11:39:08 ', ' 11:46:05 ', ' 11:45:13 ', ' 11:42:07 ', ' 11:45:22 ', ' 11:45:15 ', ' 11:35:24 ', ' 11:37:07 ', ' 11:36:47 ', ' 11:37:03 ', ' 11:45:19 ', ' 11:46:13 ', ' 11:38:52 ', ' 11:46:14 ', ' 11:41:53 ', ' 11:41:39 ', ' 11:37:10 ', ' 11:38:33 ', ' 11:42:16 ', ' 11:38:16 ', ' 11:36:53 ', ' 11:37:00 ', ' 11:43:00 ', ' 11:45:23 ', ' 11:43:05 ', ' 11:45:53 ', ' 11:42:19 ', ' 11:42:49 ', ' 11:46:39 ', ' 11:42:41 ', ' 11:38:15 ', ' 11:46:11 ', ' 11:42:57 ', ' 11:46:18 ', ' 11:37:04 ', ' 11:38:47 ', ' 11:36:59 ', ' 11:46:15 ', ' 11:46:42 ', ' 11:41:47 ', ' 11:46:22 ', ' 11:42:59 ', ' 11:38:34 ', ' 11:39:17 ', ' 11:36:29 '}

In [46]:
len(ground_truth_timestamp_coordinates)

336

In [47]:
def check(result_list):
    check_set = set(result_list)
    true,false = 0,0
    for elt in check_set:
        if elt in ground_truth_timestamp_coordinates:
            true+=1
        else:
            false+=1
    return true, false


#Testing it a in a few different ways for box plot data generation

In [57]:
width = 150
depth = 100
epsilon_list = [2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
run_number = [1]
final_result_dictionary1_list = []
for i in range(1):
    final_result_dictionary1 = {}
    for eps in epsilon_list:
        for run in run_number:
            tracker, result_list = driver_code(width, depth, eps, run, 1)
            final_true_false_count = []
            for small_list in tracker:
                final_true_false_count.append(check(small_list))
            final_result_dictionary1[(eps,run)] = check(tracker[-1])
    final_result_dictionary1_list.append(final_result_dictionary1)
    print(final_result_dictionary1)
final_result_dictionary1_list

{(2.5, 1): (70, 0), (3, 1): (52, 0), (3.5, 1): (44, 0), (4, 1): (44, 0), (4.5, 1): (36, 0), (5, 1): (23, 0), (5.5, 1): (22, 0), (6, 1): (21, 0), (6.5, 1): (10, 0), (7, 1): (13, 0)}


[{(2.5, 1): (70, 0),
  (3, 1): (52, 0),
  (3.5, 1): (44, 0),
  (4, 1): (44, 0),
  (4.5, 1): (36, 0),
  (5, 1): (23, 0),
  (5.5, 1): (22, 0),
  (6, 1): (21, 0),
  (6.5, 1): (10, 0),
  (7, 1): (13, 0)}]

In [58]:
epsilon_list = [2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
epsilon_list5 = []
epsilon_list10 = []
for eps in epsilon_list:
    epsilon_list5.append(eps/5)
    epsilon_list10.append(eps/10)

In [53]:
epsilon_list5

[0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]

In [54]:
epsilon_list10

[0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]

In [60]:
width = 150
depth = 100
run_number = [5]
final_result_dictionary5_list = []
for i in range(1):
    final_result_dictionary5 = {}
    for eps in epsilon_list5:
        for run in run_number:
            tracker, result_list = driver_code(width, depth, eps, run, 1)
            final_result_dictionary5[(eps,run)] = check(tracker[-1])
    final_result_dictionary5_list.append(final_result_dictionary5)
    print(final_result_dictionary5)
final_result_dictionary5_list    

{(0.5, 5): (207, 0), (0.6, 5): (208, 0), (0.7, 5): (182, 0), (0.8, 5): (182, 0), (0.9, 5): (158, 0), (1.0, 5): (150, 0), (1.1, 5): (159, 0), (1.2, 5): (153, 0), (1.3, 5): (144, 0), (1.4, 5): (148, 0)}


[{(0.5, 5): (207, 0),
  (0.6, 5): (208, 0),
  (0.7, 5): (182, 0),
  (0.8, 5): (182, 0),
  (0.9, 5): (158, 0),
  (1.0, 5): (150, 0),
  (1.1, 5): (159, 0),
  (1.2, 5): (153, 0),
  (1.3, 5): (144, 0),
  (1.4, 5): (148, 0)}]

In [55]:
width = 150
depth = 100
run_number = [10]
final_result_dictionary10_list = []
for i in range(1):
    final_result_dictionary10 = {}
    for eps in epsilon_list10:
        for run in run_number:
            tracker, result_list = driver_code(width, depth, eps, run, 1)
            final_result_dictionary10[(eps,run)] = check(tracker[-1])
    final_result_dictionary10_list.append(final_result_dictionary10)
    print(final_result_dictionary10)
final_result_dictionary10_list    

{(0.25, 10): (240, 0), (0.3, 10): (220, 0), (0.35, 10): (242, 0), (0.4, 10): (231, 0), (0.45, 10): (226, 0), (0.5, 10): (223, 0), (0.55, 10): (218, 0), (0.6, 10): (209, 0), (0.65, 10): (184, 0), (0.7, 10): (197, 0)}


[{(0.25, 10): (240, 0),
  (0.3, 10): (220, 0),
  (0.35, 10): (242, 0),
  (0.4, 10): (231, 0),
  (0.45, 10): (226, 0),
  (0.5, 10): (223, 0),
  (0.55, 10): (218, 0),
  (0.6, 10): (209, 0),
  (0.65, 10): (184, 0),
  (0.7, 10): (197, 0)}]

Tweaking the parmas to see the effects of it, starting with width

In [ ]:
width_list = list(range(100,210,10))
epsilon = .35
depth = 100
run_number = 10
output_list = []
for width in width_list:
    tracker,result_list = driver_code(width, depth, epsilon, 10, 1)
    print(check(tracker[-1]))
    output_list.append(check(tracker[-1]))
output_list

{(5512, 845): [{0: (39.9762, 116.3305, 0), 5: (39.9761, 116.3304, 5), 10: (39.9758, 116.3299, 10), 15: (39.9761, 116.3303, 15), 20: (39.9762, 116.3305, 20), 25: (39.9763, 116.3304, 25), 30: (39.9763, 116.3303, 30), 35: (39.9761, 116.3301, 35), 40: (39.9753, 116.3308, 40), 45: (39.9753, 116.331, 45), 50: (39.9752, 116.3313, 50)}, [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]], (5512, 846): [{55: (39.9752, 116.3335, 55), 60: (39.9752, 116.3343, 60)}, [55, 60]], (5512, 847): [{65: (39.9753, 116.3375, 65), 70: (39.9753, 116.3379, 70), 75: (39.9753, 116.3382, 75), 80: (39.9753, 116.3384, 80), 85: (39.9754, 116.3385, 85), 280: (39.9766, 116.338, 280), 305: (39.9766, 116.3375, 305), 345: (39.9763, 116.3385, 345), 350: (39.9761, 116.3384, 350), 355: (39.9759, 116.3384, 355), 360: (39.9757, 116.3384, 360), 365: (39.9754, 116.3383, 365), 370: (39.9752, 116.3381, 370), 375: (39.9752, 116.3383, 375), 380: (39.9751, 116.3384, 380), 385: (39.9752, 116.3386, 385), 390: (39.9752, 116.3384, 390), 395: (39

[(12, 33),
 (13, 32),
 (12, 33),
 (11, 29),
 (10, 31),
 (10, 27),
 (11, 30),
 (12, 32),
 (13, 31),
 (14, 36),
 (12, 36)]

Tweaking Depth

In [ ]:
depth_list = list(range(50,160,10))
epsilon = .25
width = 150
run_number = 10
threshold = 1 
output_list = []
for width in width_list:
    tracker,result_list = driver_code(width, depth, epsilon, 10, threshold)
    print(check(tracker[-1]))
    output_list.append(check(tracker[-1]))
output_list

{(5512, 845): [{0: (39.9762, 116.3305, 0), 5: (39.9761, 116.3304, 5), 10: (39.9758, 116.3299, 10), 15: (39.9761, 116.3303, 15), 20: (39.9762, 116.3305, 20), 25: (39.9763, 116.3304, 25), 30: (39.9763, 116.3303, 30), 35: (39.9761, 116.3301, 35), 40: (39.9753, 116.3308, 40), 45: (39.9753, 116.331, 45), 50: (39.9752, 116.3313, 50)}, [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]], (5512, 846): [{55: (39.9752, 116.3335, 55), 60: (39.9752, 116.3343, 60)}, [55, 60]], (5512, 847): [{65: (39.9753, 116.3375, 65), 70: (39.9753, 116.3379, 70), 75: (39.9753, 116.3382, 75), 80: (39.9753, 116.3384, 80), 85: (39.9754, 116.3385, 85), 280: (39.9766, 116.338, 280), 305: (39.9766, 116.3375, 305), 345: (39.9763, 116.3385, 345), 350: (39.9761, 116.3384, 350), 355: (39.9759, 116.3384, 355), 360: (39.9757, 116.3384, 360), 365: (39.9754, 116.3383, 365), 370: (39.9752, 116.3381, 370), 375: (39.9752, 116.3383, 375), 380: (39.9751, 116.3384, 380), 385: (39.9752, 116.3386, 385), 390: (39.9752, 116.3384, 390), 395: (39

[(13, 37),
 (11, 38),
 (13, 33),
 (14, 33),
 (7, 36),
 (10, 32),
 (11, 36),
 (9, 35),
 (11, 34),
 (12, 29),
 (11, 36)]

In [ ]:
result_list[0]

(39.9527, 116.3477, 1910)

Tweaking Threshold

In [ ]:
epsilon = 2.5
width = 150
depth = 80
run_number = 10
epsilon = .25
threshold_list = [0.95,0.96,.97,.98,.99,1,1.01,1.02,1.03,1.04,1.05]
output_list = []
for t in threshold_list:
    tracker,result_list = driver_code(width, depth, epsilon, 10, t)
    output_list.append(check(tracker[-1]))
output_list

{(5512, 845): [{0: (39.9762, 116.3305, 0), 5: (39.9761, 116.3304, 5), 10: (39.9758, 116.3299, 10), 15: (39.9761, 116.3303, 15), 20: (39.9762, 116.3305, 20), 25: (39.9763, 116.3304, 25), 30: (39.9763, 116.3303, 30), 35: (39.9761, 116.3301, 35), 40: (39.9753, 116.3308, 40), 45: (39.9753, 116.331, 45), 50: (39.9752, 116.3313, 50)}, [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]], (5512, 846): [{55: (39.9752, 116.3335, 55), 60: (39.9752, 116.3343, 60)}, [55, 60]], (5512, 847): [{65: (39.9753, 116.3375, 65), 70: (39.9753, 116.3379, 70), 75: (39.9753, 116.3382, 75), 80: (39.9753, 116.3384, 80), 85: (39.9754, 116.3385, 85), 280: (39.9766, 116.338, 280), 305: (39.9766, 116.3375, 305), 345: (39.9763, 116.3385, 345), 350: (39.9761, 116.3384, 350), 355: (39.9759, 116.3384, 355), 360: (39.9757, 116.3384, 360), 365: (39.9754, 116.3383, 365), 370: (39.9752, 116.3381, 370), 375: (39.9752, 116.3383, 375), 380: (39.9751, 116.3384, 380), 385: (39.9752, 116.3386, 385), 390: (39.9752, 116.3384, 390), 395: (39

[(10, 34),
 (10, 33),
 (9, 36),
 (11, 30),
 (12, 34),
 (13, 38),
 (11, 39),
 (9, 34),
 (9, 34),
 (10, 32),
 (12, 33)]